In [2]:
%pip install --upgrade dask[complete] numpy pandas
import dask.dataframe as dd




Note: you may need to restart the kernel to use updated packages.


In [21]:
df = dd.read_csv(r"../data/*.csv",encoding='utf-8', sample=1000000, blocksize='64mb')

In [22]:
print(df.shape[0].compute(), df.shape[1])

/home/blagden/Documents/eVakeel/lib/python3.7/site-packages/dask/dataframe/io/csv.py:137: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  path_info,


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------+---------+----------+
| Column       | Found   | Expected |
+--------------+---------+----------+
| act          | float64 | int64    |
| bailable_ipc | object  | float64  |
+--------------+---------+----------+

The following columns also raised exceptions on conversion:

- bailable_ipc
  ValueError("could not convert string to float: 'depends'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'act': 'float64',
       'bailable_ipc': 'object'}

to the call to `read_csv`/`read_table`.

In [25]:
print(df)
df.shape
len(df)
df.head()

Dask DataFrame Structure:
               ddl_case_id    act  section bailable_ipc number_sections_ipc criminal
npartitions=64                                                                      
                    object  int64  float64      float64             float64    int64
                       ...    ...      ...          ...                 ...      ...
...                    ...    ...      ...          ...                 ...      ...
                       ...    ...      ...          ...                 ...      ...
                       ...    ...      ...          ...                 ...      ...
Dask Name: read-csv, 64 tasks


/home/blagden/Documents/eVakeel/lib/python3.7/site-packages/dask/dataframe/io/csv.py:137: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  path_info,


ParserError: Error tokenizing data. C error: Expected 6 fields in line 74, saw 7


In [2]:
import dask.datasets as ds
from dask import delayed, compute
from pathlib import Path  # For handling file paths
import yaml  # For reading and parsing YAML files


In [4]:

class UnifiedReader:
    def __init__(self):
        self.readers = {
            '.json': self.read_json,
            '.yaml': self.read_yaml,
            '.yml': self.read_yaml,
            '.txt': self.read_text
        }

    @delayed
    def read_json(self, file_path):
        return dd.read_json(file_path, orient='records', lines=True)

    @delayed
    def read_yaml(self, file_path):
        with open(file_path, 'r') as file:
            data = yaml.safe_load(file)
            return data

    @delayed
    def read_text(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.readlines()

    def read_file(self, file_path: str):
        ext = Path(file_path).suffix
        if ext not in self.readers:
            raise ValueError(f"Unsupported file type: {ext}")
        return self.readers[ext](file_path)

    def read_directory(self, dir_path: str):
        files = list(Path(dir_path).rglob('*'))
        tasks = [self.read_file(str(file_path)) for file_path in files if file_path.suffix in self.readers]
        return compute(*tasks)

def main():
    reader = UnifiedReader()
    data_dir = 'data'
    csv_data = reader.read_directory('../data/*.csv')
    json_data = reader.read_directory('../data/*.json')
    yaml_data = reader.read_directory('../data/*.yaml')
    txt_data = reader.read_directory('../data/*.txt')



    print("Data from all files:")
    print(json_data)

if __name__ == "__main__":
    main()


Data from all files:
()
